## Borg (Monostate)

Wzorzec podobny do Singletona, ale zamiast wymuszać jedną instancję, pozwala na wiele instancji które dzielą ten sam stan.

**Nazewnictwo:**
- **Monostate** - formalna nazwa wzorca
- **Borg** - pythonowa nazwa (od rasy Borg ze Star Trek: "you will be assimilated")

**Idea:** Wszystkie obiekty są różne, ale zachowują się jakby były tym samym obiektem (dzielą stan).

### Problem: Singleton vs potrzeba wielu instancji

Czasem chcemy:
- Wspólny stan między wszystkimi obiektami
- Ale możliwość tworzenia wielu instancji (np. dziedziczenie, duck typing)

### Podstawowa implementacja

In [1]:
class Borg:
    _shared_state = {}  # Wspólny stan dla wszystkich instancji
    
    def __init__(self):
        self.__dict__ = self._shared_state  # Wszystkie instancje dzielą ten sam __dict__

# Użycie
b1 = Borg()
b2 = Borg()

print(f"b1 is b2: {b1 is b2}")  # False - różne obiekty
print(f"id(b1): {id(b1)}")
print(f"id(b2): {id(b2)}")

b1 is b2: False
id(b1): 2981266952880
id(b2): 2981266981904


In [2]:
# Ale dzielą stan!
b1.x = 10
print(f"b1.x: {b1.x}")
print(f"b2.x: {b2.x}")  # b2 też ma x!

b2.y = 20
print(f"b1.y: {b1.y}")  # b1 też ma y!

print(f"\nb1.__dict__ is b2.__dict__: {b1.__dict__ is b2.__dict__}")  # True - ten sam dict

b1.x: 10
b2.x: 10
b1.y: 20

b1.__dict__ is b2.__dict__: True


### Jak to działa?

**Mechanizm:**
```python
self.__dict__ = self._shared_state
```

- `__dict__` przechowuje atrybuty instancji
- Zamiast tworzyć nowy `__dict__`, wszystkie instancje wskazują na ten sam `_shared_state`
- Każda zmiana atrybutu w jednym obiekcie jest widoczna we wszystkich innych

### Praktyczny przykład: konfiguracja aplikacji

In [3]:
class AppConfig:
    _shared_state = {}
    
    def __init__(self):
        self.__dict__ = self._shared_state
    
    def set_config(self, key, value):
        self.__dict__[key] = value
    
    def get_config(self, key):
        return self.__dict__.get(key)


In [4]:
# Moduł A
config_a = AppConfig()
config_a.set_config('database', 'postgresql://localhost/mydb')
config_a.set_config('debug', True)

In [5]:
# Moduł B - nowa instancja, ale ten sam stan!
config_b = AppConfig()
print(f"Database z config_b: {config_b.get_config('database')}")
print(f"Debug z config_b: {config_b.get_config('debug')}")

Database z config_b: postgresql://localhost/mydb
Debug z config_b: True


In [6]:
# Zmiana w B widoczna w A
config_b.set_config('port', 8080)
print(f"Port z config_a: {config_a.get_config('port')}")

Port z config_a: 8080


### Borg vs Singleton

In [7]:
# Singleton
class Singleton:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

s1 = Singleton()
s2 = Singleton()
print("Singleton:")
print(f"  s1 is s2: {s1 is s2}")  # True - ta sama instancja


Singleton:
  s1 is s2: True


In [8]:
# Borg
class Borg:
    _shared_state = {}
    def __init__(self):
        self.__dict__ = self._shared_state

b1 = Borg()
b2 = Borg()
print("\nBorg:")
print(f"  b1 is b2: {b1 is b2}")  # False - różne instancje
b1.x = 10
print(f"  b1.x == b2.x: {b1.x == b2.x}")  # True - ten sam stan


Borg:
  b1 is b2: False
  b1.x == b2.x: True


| Aspekt | Singleton | Borg |
|--------|-----------|------|
| Instancje | Jedna | Wiele |
| `obj1 is obj2` | True | False |
| Stan | Wspólny (bo jedna instancja) | Wspólny (dzielony `__dict__`) |
| Dziedziczenie | Problematyczne | Naturalne |
| Duck typing | Problematyczne | Działa |
| Cel | Jedna instancja | Wspólny stan |

### Dziedziczenie

#### Bez nadpisywania - dziecko dzieli stan z rodzicem

In [ ]:
class Borg:
    _shared_state = {}
    def __init__(self):
        self.__dict__ = self._shared_state

class BorgChild(Borg):
    pass  # NIE nadpisujemy _shared_state

# Test
parent = Borg()
child = BorgChild()

parent.x = 100
print(f"child.x: {child.x}")  # 100 - dzielą stan!
print(f"parent._shared_state is child._shared_state: {parent._shared_state is child._shared_state}")

#### Z nadpisywaniem - osobny stan

In [ ]:
class Borg:
    _shared_state = {}
    def __init__(self):
        self.__dict__ = self._shared_state

class BorgChild(Borg):
    _shared_state = {}  # NADPISUJEMY - własny stan!

# Test
parent = Borg()
child = BorgChild()

parent.x = 100
print(f"hasattr(child, 'x'): {hasattr(child, 'x')}")  # False - NIE dzielą stanu
print(f"parent._shared_state is child._shared_state: {parent._shared_state is child._shared_state}")

### Warianty implementacji

#### Wariant 1: __new__ (bardziej pythoniczny)

In [10]:
class BorgNew:
    _shared_state = {}
    
    def __new__(cls, *args, **kwargs):
        obj = super().__new__(cls)
        obj.__dict__ = cls._shared_state
        return obj


In [11]:
# Użycie
b1 = BorgNew()
b2 = BorgNew()

b1.value = 42
print(f"b2.value: {b2.value}")  # 42

b2.value: 42


#### Wariant 2: Dekorator

In [12]:
def borg(cls):
    """Dekorator zamieniający klasę w Borg"""
    cls._shared_state = {}
    original_init = cls.__init__
    
    def new_init(self, *args, **kwargs):
        self.__dict__ = self._shared_state
        original_init(self, *args, **kwargs)
    
    cls.__init__ = new_init
    return cls


In [13]:
@borg
class MyConfig:
    def __init__(self, name):
        if not hasattr(self, 'initialized'):
            self.name = name
            self.initialized = True


In [15]:
# Użycie
c1 = MyConfig("config1")
c2 = MyConfig("config2")  # name nie zmieni się, bo już zainicjowane

print(f"c1.name: {c1.name}")
print(f"c2.name: {c2.name}")  # config1 - dzielą stan!

c1.name: config1
c2.name: config1


### Uwaga przy __init__

Gdy używasz Borg, `__init__` jest wywoływane dla KAŻDEJ nowej instancji, więc może nadpisywać atrybuty!

In [18]:
class BorgProblem:
    _shared_state = {}
    
    def __init__(self, name):
        self.__dict__ = self._shared_state
        self.name = name  # Każda nowa instancja nadpisuje name!


In [19]:
b1 = BorgProblem("First")
print(f"b1.name: {b1.name}")

b1.name: First


In [20]:
b2 = BorgProblem("Second")
print(f"b1.name po utworzeniu b2: {b1.name}")  # Second - nadpisane!
print(f"b2.name: {b2.name}")

b1.name po utworzeniu b2: Second
b2.name: Second


#### Propozycja rozwiązania: sprawdzaj czy już zainicjowane

In [25]:
class BorgFixed:
    _shared_state = {}
    
    def __init__(self, name):
        self.__dict__ = self._shared_state
        
        # Inicjalizuj tylko raz
        if not hasattr(self, 'initialized'):
            self.name = name
            self.initialized = True


In [26]:
b1 = BorgFixed("First")
print(f"b1.name: {b1.name}")


b1.name: First


In [27]:
b2 = BorgFixed("Second")  # name nie zmieni się
print(f"b1.name: {b1.name}")  # First - nie nadpisane
print(f"b2.name: {b2.name}")  # First - dzielą stan

b1.name: First
b2.name: First


### Zalety Borg

#### 1. Dziedziczenie działa naturalnie

In [29]:
class ConfigBase:
    _shared_state = {}
    def __init__(self):
        self.__dict__ = self._shared_state

class DatabaseConfig(ConfigBase):
    def set_db_url(self, url):
        self.db_url = url

class CacheConfig(ConfigBase):
    def set_cache_ttl(self, ttl):
        self.cache_ttl = ttl

# Wszystkie dzieci dzielą ten sam stan
db = DatabaseConfig()
cache = CacheConfig()

db.set_db_url("postgresql://localhost")
cache.set_cache_ttl(3600)

print(f"db.cache_ttl: {db.cache_ttl}")  # Dostępne w db!
print(f"cache.db_url: {cache.db_url}")  # Dostępne w cache!

db.cache_ttl: 3600
cache.db_url: postgresql://localhost


### Wady Borg

#### 1. Mylące - wygląda jak osobne obiekty

In [30]:
class Counter:
    _shared_state = {}
    def __init__(self):
        self.__dict__ = self._shared_state
        if not hasattr(self, 'count'):
            self.count = 0
    
    def increment(self):
        self.count += 1

c1 = Counter()
c2 = Counter()

# Wygląda jak dwa osobne liczniki, ale...
c1.increment()
print(f"c1.count: {c1.count}")
print(f"c2.count: {c2.count}")  # Też 1 - mylące!

c1.count: 1
c2.count: 1


#### 2. `__init__` wywoływane wielokrotnie

In [ ]:
# Trzeba pamiętać o sprawdzaniu initialized
# Łatwo o błąd nadpisywania atrybutów

### Kiedy używać Borg?

**Używaj gdy:**
- Potrzebujesz wspólnego stanu między obiektami
- Dziedziczenie jest ważne

**Nie używaj gdy:**
- Moduł z instancją wystarczy (pythoniczny idiom)
- Wspólny stan nie jest wymagany

**W praktyce:**
W Pythonie rzadko potrzebujesz Borg - najczęściej wystarczy moduł z instancją lub Singleton.

### Podsumowanie

**Borg (Monostate):**
- Wiele instancji, jeden wspólny stan
- Mechanizm: `self.__dict__ = cls._shared_state`
- Wszystkie zmiany w jednym obiekcie widoczne we wszystkich

**Borg vs Singleton:**
- Singleton: jedna instancja (`obj1 is obj2`)
- Borg: wiele instancji, jeden stan (`obj1.x == obj2.x`)

**Dziedziczenie:**
- Bez nadpisania: dziecko dzieli stan z rodzicem
- Z nadpisaniem `_shared_state`: osobny stan

**Uwagi:**
- `__init__` wywoływane wielokrotnie - sprawdzaj `initialized`
- Może być mylące (wygląda jak osobne obiekty)
- W Pythonie rzadko potrzebne (moduł często wystarczy)